# Проект

В данном проекте используется датасет большинства экзопланет, которые были найдены космической обсерваторией Кеплер.

Данные о планетах записаны в файле cumulative.csv

Задача - предсказать значение колонки koi_score

Расшифровка столбцов в cumulative.csv для справки: (вода для нуждающихся)

Здесь:
KOI - Kepler Object of Interest - Объект, интересующий Кеплер
KIC - Kepler Input Catalogue - входной каталог объектов Кеплер

Наименования объектов:
- rowid - порядковый номер объекта в таблице
- kepid - Идентификационный номер цели, указанный в входном каталоге Kepler input Catalog (KIC)
- kepoi_name - Число, используемое для идентификации и отслеживания интересующего объекта Кеплера (KOI)
- kepler_name - Имя числа Kepler в форме "Kepler-N", плюс строчная буква, идентифицирующая планету.
- koi_tce_plnt_num - TCE номер планеты, объединенный с KOI.
- koi_tce_delivname - Имя доставки TCE, соответствующее данным TCE, объединенным с KOI.

(Данные колонки вообще не влияют на значение koi_score, ведь влияются всего лишь идентификаторами объектов)

Типы объектов:
- koi_disposition - Категория этого кои из архива экзопланет. Текущие значения являются кандидатами (CANDIDATE), ложноположительными (FALSE POSITIVE), не диспозицированными (NOT DISPOSITIONED) и подтвержденными (CONFIRMED).
- koi_pdisposition - Флаг, обозначающий наиболее вероятное физическое объяснение KOI. Типичными значениями являются FALSE POSITIVE, NOT DISPOSITIONED, и CANDIDATE.
- koi_score - Значение между 0 и 1, которое указывает на уверенность в расположении KOI. Для CANDIDATE более высокое значение указывает на большую уверенность в его расположении, в то время как для FALSE POSITIVE более высокое значение указывает на меньшую уверенность в этом расположении.
- koi_fpflag_nt - KOI, чья кривая света не согласуется с кривой света транзитной планеты.
- koi_fpflag_ss - KOI, который, как наблюдают, имеет значительное вторичное событие, транзитную форму или изменчивость вне затмения, которая указывает, что транзитное событие, скорее всего, этим событием.
- koi_fpflag_co - Источник сигнала исходит от ближайшей звезды, о чем можно судить по измерению центроидного положения изображения как в транзите, так и вне его, или по силе транзитного сигнала во внешних пикселях цели (гало) по сравнению с транзитным сигналом от пикселов в оптимальной (или основной) апертуре.
- koi_fpflag_ec - KOI разделяет тот же период и эпоху, что и другой объект, и считается результатом загрязнения потока в Апертуре или электронных перекрестных помех.

(Данные колонки влияют на результат и будут использованы в дальнейшем)

Свойства объектов:
Свойства транзитов:
- koi_period - Интервал между последовательными планетарными транзитами.
- koi_time0bk - Время, соответствующее центру первого обнаруженного транзита в Барицентрическом юлианском дне (BJD) минус постоянное смещение в 2454 833,0 дня.
- koi_duration- Продолжительность наблюдаемых транзитов. Продолжительность измеряется от первого контакта между планетой и звездой до последнего контакта.
- koi_depth - Доля звездного света, которая теряется при минимуме планетарного транзита.
- koi_model_snr - Глубина транзита, нормируемая на среднюю неопределенность потока во время транзитов.

(Данные колонки влияют на результат и поэтому предсказания базируются на них)

Свойства планет:
- koi_prad - Радиус планеты. Планетарный радиус- это произведение отношения радиуса звезды планеты и звездного радиуса.
- koi_teq - Аппроксимация для температуры планеты.
- koi_insol - Инсоляционный поток - это еще один способ придать равновесную температуру. Она зависит от параметров звезды (в частности, от ее радиуса и температуры), а также от полуоси планеты. Она дана в единицах относительно тех, что измерены для Земли от Солнца.
- koi_impact - Небо-проецируемое расстояние между центром звездного диска и центром диска планеты при соединении, нормированное на звездный радиус.

(Данные колонки влияют на результат и поэтому предсказания базируются на них)

Свойства звёзд:
- koi_steff - Фотосферная температура звезды
- koi_slogg - 10-ный логарифм ускорения , обусловленного гравитацией на поверхности звезды.
- koi_srad - Фотосферный радиус звезды

(Данные колонки на мой взгляд не влияют (или влияют, но незначительно) на результат, и поэтому они возможно будут использоваться в дальнейшем)

Координаты объекта:
- ra - KIC Right Ascension
- dec - KIC Declination

(Опять же это просто местоположение объекта, потому это не влияет на результат)

Колонка, суть которой мне не удалось понять(а потому не будет использоваться в моделях):
- koi_kepmag - Kepler-band (mag)

In [62]:
import numpy as np, pandas as pd, csv, random
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

dataf = pd.read_csv('C:/Users/lilpu/OneDrive/Рабочий стол/cumulative.csv')  
dataf.index = range(9564)


# ['koi_disposition', 'koi_period', 'koi_duration', 'koi_depth', 'koi_model_snr', 'koi_prad', 'koi_teq', 'koi_impact']
#                                              ^^^
# те параметры, по которым я хочу определять koi_score. koi_disposition пока что не используется.

col = ['koi_score', 'koi_period', 'koi_duration', 'koi_depth', 'koi_model_snr', 'koi_prad', 'koi_teq', 'koi_impact'];
df = pd.DataFrame()
for i in col:
    df[i] = dataf[i]
# избавляемся от строк с NaN 
df.dropna(inplace=True)


x = df[col[1:]].to_numpy()
y = df['koi_score'].to_numpy()
# dataf.index = range(y.size)
df[['koi_score', 'koi_period', 'koi_duration', 'koi_depth', 'koi_model_snr', 'koi_prad', 'koi_teq']]
# print(y)
# print(X)

,koi_score,koi_period,koi_duration,koi_depth,koi_model_snr,koi_prad,koi_teq
0,1.000,9.488036,2.95750,615.8,35.8,2.26,793.0
1,0.969,54.418383,4.50700,874.8,25.8,2.83,443.0
2,0.000,19.899140,1.78220,10829.0,76.3,14.60,638.0
3,0.000,1.736952,2.40641,8079.2,505.6,33.46,1395.0
4,1.000,2.525592,1.65450,603.3,40.9,2.75,1406.0
...,...,...,...,...,...,...,...
9559,0.000,8.589871,4.80600,87.7,8.4,1.11,929.0
9560,0.000,0.527699,3.22210,1579.2,453.3,29.35,2088.0
9561,0.497,1.739849,3.11400,48.5,10.6,0.72,1608.0
9562,0.021,0.681402,0.86500,103.6,12.3,1.07,2218.0


In [73]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state = 69)


# создание модели
model = linear_model.LinearRegression()
model.fit(x_train, y_train)

# вывод результата
RMSE = mean_squared_error(y_test, model.predict(x_test), squared=False)
print("RMSE =", RMSE)

RMSE = 0.4021557084949412
